In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing import image
import numpy as np
import pandas as pd
import os


Using TensorFlow backend.


In [2]:
class myCallback(tf.keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('acc') > 0.98):   
            print("\nReached %2.2f%% accuracy, so stopping training!!" %(0.98*100))   
            self.model.stop_training = True
callback= myCallback()

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    directory = "../data/train",
    target_size=(150, 150),
    batch_size=8,
    class_mode="categorical")


validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    directory = '../data/test/',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

Found 4158 images belonging to 3 classes.
Found 600 images belonging to 4 classes.


In [4]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),

    tf.keras.layers.Dense(4, activation='sigmoid')
])

In [5]:
model.compile(loss='mean_squared_error',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

In [7]:
model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=100,
    callbacks=[callback]) # callbacks=[callback] (use it in the fit_generator)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 100 steps, validate for 100 steps
Epoch 1/20
  1/100 [..............................] - ETA: 31s

TypeError: '>' not supported between instances of 'NoneType' and 'float'

In [ ]:
evaluate=model.evaluate_generator(generator=validation_generator,steps=300)
evaluate

In [ ]:
def find_class(result_list):
    if result_list[0] == 1 :
        return "Bacteria"
    elif result_list[1] == 1 : 
        return "Covid"
    elif result_list[2] == 1 : 
        return "Normal"
    elif result_list[3] == 1 : 
        return "Virus"
    else:
        return "Sınıf Tespit edilemedi"

In [ ]:
basepath = '../data/predict/'
for entry in os.listdir(basepath):
    if os.path.isfile(os.path.join(basepath, entry)):
        print(entry)
        img = image.load_img(os.path.join(basepath, entry), target_size=(150, 150))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)

        images = np.vstack([x])
        classes = model.predict(images, batch_size=10)
        print("Tahmin = " + find_class(classes[0]))
        print("XXXXXXXXXXXXXXXXXXXXXXXXXXX")
        

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // BATCH_SIZE,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // BATCH_SIZE,
    epochs = EPOCHS)



In [ ]:

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:


print("training_accuracy", history.history['accuracy'][-1])
print("validation_accuracy", history.history['val_accuracy'][-1])


In [ ]:

label = validation_generator.classes


In [ ]:

pred= model.predict(validation_generator)
predicted_class_indices=np.argmax(pred,axis=1)
labels = (validation_generator.class_indices)
labels2 = dict((v,k) for k,v in labels.items())
predictions = [labels2[k] for k in predicted_class_indices]


In [ ]:

from sklearn.metrics import confusion_matrix

cf = confusion_matrix(predicted_class_indices,label)
cf

In [ ]:


exp_series = pd.Series(label)
pred_series = pd.Series(predicted_class_indices)
pd.crosstab(exp_series, pred_series, rownames=['Actual'], colnames=['Predicted'],margins=True)


In [ ]:

plt.matshow(cf)
plt.title('Confusion Matrix Plot')
plt.colorbar()
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show();

